In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, NMF, LatentDirichletAllocation
from umap import UMAP

/home/rainergo/.local/share/virtualenvs/RELATION-EXTRACTION-Yry67amm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [3]:
d1 = ['alles hat ein ende']
d2 = ['alles hat einen anfang']
d3 = [[1,2,3,4,5]]
d4 = [[1,2,3,4,5]]
df = pd.DataFrame(data={'d':[d1, d2]})
df

,d
0,[alles hat ein ende]
1,[alles hat einen anfang]


In [4]:
vocab = {val:ind for ind, val in enumerate(sorted(['alles', 'hat', 'ein', 'ende', 'einen', 'anfang']))}
vocab

{'alles': 0, 'anfang': 1, 'ein': 2, 'einen': 3, 'ende': 4, 'hat': 5}

In [5]:
vectorizer.fit(vocab)

TfidfVectorizer()

In [6]:
df['vector']: np.ndarray = df.apply(lambda x: vectorizer.transform(x.d).toarray() if x.d is not None else [], axis=1)
df

,d,vector
0,[alles hat ein ende],"[[0.5, 0.0, 0.5, 0.0, 0.5, 0.5]]"
1,[alles hat einen anfang],"[[0.5, 0.5, 0.0, 0.5, 0.0, 0.5]]"


In [98]:
df['reduce_dim'] = df.apply(lambda x: np.squeeze(x.vector), axis=1)
df

,d,vector,vector_scaled,red_vector,red_vector_unscaled,reduce_dim
0,[alles hat ein ende],"[[0.5, 0.0, 0.5, 0.0, 0.5, 0.5]]","[[0.0, -1.0, 1.0, -1.0, 1.0, 0.0]]","[[-2.0, 0.0]]","[[-0.5, 0.5]]","[0.5, 0.0, 0.5, 0.0, 0.5, 0.5]"
1,[alles hat einen anfang],"[[0.5, 0.5, 0.0, 0.5, 0.0, 0.5]]","[[0.0, 1.0, -1.0, 1.0, -1.0, 0.0]]","[[2.0, 0.0]]","[[0.5, 0.5]]","[0.5, 0.5, 0.0, 0.5, 0.0, 0.5]"


In [7]:
all_vecs = np.vstack(df.vector[df.vector.str.len() != 0])
print(type(all_vecs))
print(all_vecs.shape)
all_vecs

<class 'numpy.ndarray'>
(2, 6)


array([[0.5, 0. , 0.5, 0. , 0.5, 0.5],
       [0.5, 0.5, 0. , 0.5, 0. , 0.5]])

In [12]:
scaler = StandardScaler()
scaled_vecs =scaler.fit_transform(all_vecs)
print(scaled_vecs.shape)
scaled_vecs

(2, 6)


array([[ 0., -1.,  1., -1.,  1.,  0.],
       [ 0.,  1., -1.,  1., -1.,  0.]])

In [9]:
df['vector_scaled']: np.ndarray = df.apply(lambda x: scaler.transform(x.vector) if x.vector is not None else [], axis=1)
df

,d,vector,vector_scaled
0,[alles hat ein ende],"[[0.5, 0.0, 0.5, 0.0, 0.5, 0.5]]","[[0.0, -1.0, 1.0, -1.0, 1.0, 0.0]]"
1,[alles hat einen anfang],"[[0.5, 0.5, 0.0, 0.5, 0.0, 0.5]]","[[0.0, 1.0, -1.0, 1.0, -1.0, 0.0]]"


In [16]:
reduction_model = PCA(n_components=2, svd_solver='auto')
reduction_model.fit(scaled_vecs)

PCA(n_components=2)

In [17]:
df['red_vector'] = df.apply(lambda x: reduction_model.transform(x.vector_scaled) if x.vector_scaled is not None else [], axis=1)
df

,d,vector,vector_scaled,red_vector
0,[alles hat ein ende],"[[0.5, 0.0, 0.5, 0.0, 0.5, 0.5]]","[[0.0, -1.0, 1.0, -1.0, 1.0, 0.0]]","[[-2.0, 0.0]]"
1,[alles hat einen anfang],"[[0.5, 0.5, 0.0, 0.5, 0.0, 0.5]]","[[0.0, 1.0, -1.0, 1.0, -1.0, 0.0]]","[[2.0, 0.0]]"


In [18]:
df['red_vector_unscaled'] = df.apply(lambda x: reduction_model.transform(x.vector) if x.vector is not None else [], axis=1)
df

,d,vector,vector_scaled,red_vector,red_vector_unscaled
0,[alles hat ein ende],"[[0.5, 0.0, 0.5, 0.0, 0.5, 0.5]]","[[0.0, -1.0, 1.0, -1.0, 1.0, 0.0]]","[[-2.0, 0.0]]","[[-0.5, 0.5]]"
1,[alles hat einen anfang],"[[0.5, 0.5, 0.0, 0.5, 0.0, 0.5]]","[[0.0, 1.0, -1.0, 1.0, -1.0, 0.0]]","[[2.0, 0.0]]","[[0.5, 0.5]]"


In [61]:
d3 = [1,2,3,4,5]
d4 = [1,2,3,4,5]
dff = pd.DataFrame(data={'d':[d3, d4]})
dff

,d
0,"[1, 2, 3, 4, 5]"
1,"[1, 2, 3, 4, 5]"


In [41]:
res = dff.d.values
print(res.shape)
print(type(res))
res

(2,)
<class 'numpy.ndarray'>


array([list([1, 2, 3, 4, 5]), list([1, 2, 3, 4, 5])], dtype=object)

In [51]:
dff['expanded'] = dff.d.apply(lambda x: np.array(x, ndmin=2))

# np.array(X, ndmin=2)
dff

,d,expanded
0,"[1, 2, 3, 4, 5]","[[1, 2, 3, 4, 5]]"
1,"[1, 2, 3, 4, 5]","[[1, 2, 3, 4, 5]]"


In [78]:
d3 = [1,2,3,4,5]
d_none = pd.NA
d4 = [1,2,3,4,5]
dfff = pd.DataFrame(data={'d':[d3, d_none, d4]})
dfff

,d
0,"[1, 2, 3, 4, 5]"
1,<NA>
2,"[1, 2, 3, 4, 5]"


In [89]:
dfff['test_na'] = dfff.d.apply(lambda x: np.array(x, ndmin=2) if x is not pd.NA else pd.NA)
dfff

,d,test_na
0,"[1, 2, 3, 4, 5]","[[1, 2, 3, 4, 5]]"
1,<NA>,<NA>
2,"[1, 2, 3, 4, 5]","[[1, 2, 3, 4, 5]]"


In [92]:
res = np.vstack(dfff.test_na[~dfff.test_na is not pd.NA])
res

KeyError: True

In [80]:
# np.vstack(dfff.d[dfff.d != None])
# dfff.d[dfff.d != None]
np.vstack(dfff[~dfff.d.isna()].values)

array([[list([1, 2, 3, 4, 5])],
       [list([1, 2, 3, 4, 5])]], dtype=object)

In [53]:
my_list = [None, None, ['konzernergebnis', 'starten', 'leicht', 'wachstum', 'kostenreduktion'], ['vorstandsvorsitzender', 'hypoport'], ['gegeben', 'rahmenbedingung', 'erfolgreich', 'starten'], None, ['zahlungsmittel', 'zahlungsmittelaequivalente', 'boersenfaehig', 'wertpapier', 'geschaeftsaktivitaet', 'jahreshaelft', 'sichern'], ['v', 'unternehmen', 'bereich', 'klinisch', 'onkologie', 'innovativ', 'multispezifisch', 'antikoerper', 'volllaeng', 'entwickeln', 'finanzergebnis', 'quartal', 'veroeffentlichen', 'update', 'geschaeftsentwicklung', 'vorlegen'], ['merus', 'planen', 'halbjahr', 'klinisch', 'datum', 'veroeffentlichen'], ['merus', 'untersuchen', 'kombination', 'afatinib', 'patient', 'nsclc'], ['strich', 'bereinigen', 'sondereffekt', 'abschreibung', 'wertminderung', 'fuer', 'gewinn', 'vorweisen', 'mittlerer', 'zweistellig', 'million', 'euro', 'bereich', 'vorweisen', 'vorjahreswert', 'million', 'euro', 'liegen'], ['vorjahreswert', 'konzernebene', 'debatte', 'korrekt', 'bilanzierung', 'gutscheinverkaeufe', 'schweizer', 'anpassen'], ['spannung', 'unterdessen', 'erwarten', 'einfluss', 'ausweiten'], ['kandidatur', 'ex', 'chefin', 'mittwoch', 'offiziell', 'geben'], None, None, None, None, None, None, None, ['exasol', 'veroeffentlicht', 'testiert', 'konzernabschluss', 'ergebnis']]

In [59]:
# [s for l in my_list for s in l if l]
[s for l in [li for li in my_list if li is not None] for s in l]

['konzernergebnis',
 'starten',
 'leicht',
 'wachstum',
 'kostenreduktion',
 'vorstandsvorsitzender',
 'hypoport',
 'gegeben',
 'rahmenbedingung',
 'erfolgreich',
 'starten',
 'zahlungsmittel',
 'zahlungsmittelaequivalente',
 'boersenfaehig',
 'wertpapier',
 'geschaeftsaktivitaet',
 'jahreshaelft',
 'sichern',
 'v',
 'unternehmen',
 'bereich',
 'klinisch',
 'onkologie',
 'innovativ',
 'multispezifisch',
 'antikoerper',
 'volllaeng',
 'entwickeln',
 'finanzergebnis',
 'quartal',
 'veroeffentlichen',
 'update',
 'geschaeftsentwicklung',
 'vorlegen',
 'merus',
 'planen',
 'halbjahr',
 'klinisch',
 'datum',
 'veroeffentlichen',
 'merus',
 'untersuchen',
 'kombination',
 'afatinib',
 'patient',
 'nsclc',
 'strich',
 'bereinigen',
 'sondereffekt',
 'abschreibung',
 'wertminderung',
 'fuer',
 'gewinn',
 'vorweisen',
 'mittlerer',
 'zweistellig',
 'million',
 'euro',
 'bereich',
 'vorweisen',
 'vorjahreswert',
 'million',
 'euro',
 'liegen',
 'vorjahreswert',
 'konzernebene',
 'debatte',
 'kor

In [96]:
test_points = np.random.random(size=(1, 5))
test_points.shape

(1, 5)